In [12]:
from pygooglenews import GoogleNews
import pandas as pd
pd.set_option('display.max_colwidth', None)
from newspaper import Article
import nltk
import time
from tqdm import tqdm
from urllib.parse import urlparse, parse_qs
import sqlalchemy
from datetime import datetime,date
from sqlalchemy import create_engine
from fuzzywuzzy import fuzz,process
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import json
from bs4 import BeautifulSoup
from time import sleep
import spacy
import re
from string import punctuation
from collections import Counter
from string import punctuation
import warnings
import tqdm
warnings.filterwarnings("ignore")
import datetime
from dateutil import parser
from datetime import datetime as dt
from datetime import timedelta as td
import sqlite3
from random import randint
from time import sleep

In [2]:
webdriver_path = r'C:\Users\LTIM_10700357\Downloads\chromedriver_win32\chromedriver.exe'  # Adjust this path

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode

# Initialize WebDriver
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 129.0.6668.90 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006BA813+48355]
	(No symbol) [0x0064C4B1]
	(No symbol) [0x00555358]
	(No symbol) [0x005761AC]
	(No symbol) [0x00571EF3]
	(No symbol) [0x00570579]
	(No symbol) [0x005A0C55]
	(No symbol) [0x005A093C]
	(No symbol) [0x0059A536]
	(No symbol) [0x005782DC]
	(No symbol) [0x005793DD]
	GetHandleVerifier [0x0091AABD+2539405]
	GetHandleVerifier [0x0095A78F+2800735]
	GetHandleVerifier [0x0095456C+2775612]
	GetHandleVerifier [0x007451E0+616112]
	(No symbol) [0x00655F8C]
	(No symbol) [0x00652328]
	(No symbol) [0x0065240B]
	(No symbol) [0x00644FF7]
	BaseThreadInitThunk [0x76C900C9+25]
	RtlGetAppContainerNamedObjectPath [0x76FF7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x76FF7B1E+238]


In [ ]:
# today = datetime.datetime.today().strftime('%Y-%m-%d')
# today1 = dt.now() # get date and time today
# delta = td(days=7) #initialize delta
# date_before = today1 - delta # add the delta days
# date_before=date_before.strftime('%Y-%m-%d') # format it
print('date taken')
# today_new = parser.parse(today)
# date_before_new=parser.parse(date_before)

# current_date = today_new.isoformat()
# previous_date = date_before_new.isoformat()
# week_later_date = week_later.isoformat()
today = datetime.datetime.utcnow().today().strftime('%Y-%m-%d')
today1 = dt.utcnow().now() # get date and time today
delta = td(days=7) #initialize delta
date_before = today1 - delta # add the delta days
date_before=date_before.strftime('%Y-%m-%d') # format it

In [ ]:
date_dict={'name': ['February_March_2023'],'from':[date_before],'to':[today],'status':[''],'completed_on':['']}
# date_dict={'name': ['February_March_2023'],'from':['2024-08-16'],'to':['2024-08-23'],'status':[''],'completed_on':['']}

In [ ]:
# date_dict={'name': ['February_March_2023'],'from':['17-04-2023'],'to':['23-04-2023'],'status':[''],'completed_on':['']}

In [ ]:
#date_range=pd.read_csv(r'C:\Users\SAL006\FATF\date_range.csv')
date_range=pd.DataFrame.from_dict(date_dict)

In [ ]:
#date_range=date_range.sort_values('to', ascending=False)
#date_range['status']=''
#date_range['completed_on']=''
# date_range['from'] = date_range['from'].dt.date
# date_range['to'] = date_range['to'].dt.date

In [ ]:
date_range.head()

In [ ]:
gn = GoogleNews(lang = 'en', country = 'IN')

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
def get_titles(search_string,from_date,to_date):
    #print('*Fetching URLs*')
    stories=[]
    search=gn.search(search_string,from_=from_date,to_=to_date)
    newsitems=search['entries']
    # print(newsitems)
    for item in newsitems:
        story={
            'published': item.published,
            'title': item.title,
            'link': item.link,
            'searchcriteria': search_string,
            'source':item['source']['href']
           # 'pub':item['source']['title']
        }
        stories.append(story)
    return stories

In [ ]:
def get_hotwords(text, top_n=10, min_length=4):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    
    # Create a list to store tokens
    tokens = [token.text for token in doc 
              if token.text not in nlp.Defaults.stop_words and 
              token.text not in punctuation and 
              token.pos_ in pos_tag and 
              len(token.text) >= min_length]
    
    # Count the frequency of each token
    token_freq = Counter(tokens)
    
    # Get the most common tokens
    most_common_tokens = token_freq.most_common(top_n)
    
    # Extract the tokens from the tuples
    result = [token for token, freq in most_common_tokens]
    
    return result

In [4]:
def fetch_article(df_news):
    article_list=[]
    for i in df_news.to_dict(orient='records'):
        try:
            url=i['link']
            source=i['source']
            pub=source.split('//')[1]
            article = Article(url)
            article.download()
            article.parse()
            authors=article.authors
            # atext=re.sub('(Also watch|Also read).*$','',article.text)
            # atext=article.text
            sleep(randint(1,10))
            atext=fetch_article_body(url)
            # print(atext[:200])
            atitle=i['title']
            article.nlp()
            hotwords = set(get_hotwords(atext))
            keywords1 = ', '.join(sorted(hotwords))
            words_list = keywords1.split(',')
            selected_words = random.sample(words_list, 20)
            keywords = ','.join(selected_words)
            # keywords=article.keywords #finds significant keywords
            persons=person_tagger(atext)
            persons_cleaned=[]
            for person in persons:
                if len(nltk.word_tokenize(person))>1:
                    persons_cleaned.append(person.upper())        
            article = {'publish_date': i['published'],
                    'publisher': regexp.match(url).group(1).upper(),
                    'pub':pub,
                    'title': atitle,
                    'article': atext.replace('\n',''),
                    'persons': persons_cleaned,
                    'keywords': keywords,            
                    'link': url,
                    'source':source,
                    'searchcriteria': i['searchcriteria']
                    }
            # print(article)
            # print('-------------------------')
            article_list.append(article)
            # print(article_list)
        except Exception as e:
            pass
    # print(article_list)
    return article_list

In [5]:
article_body1 = 'No Records Extracted'
def fetch_article_body(link):
    webdriver_path = r'C:\Users\LTIM_10700357\Downloads\chrome-win64\chrome-win64\chrome.exe'  # Adjust this path
# Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
# Initialize WebDriver
    service = Service(webdriver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    article_url = link
    driver.get(article_url)
    sleep(randint(10,100))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
# print(soup.prettify())
    scripts = soup.find_all('script', type='application/ld+json')
    driver.quit()
    for script in scripts:
        if script:
            json_ld_data = script.string
            json_ld_data = json_ld_data.strip()  # Remove leading/trailing whitespace
            json_ld_data = json_ld_data.replace('\n', '')  # Remove newlines
            json_ld_data = json_ld_data.replace('  ', '')
            try:
                data = json.loads(json_ld_data)
                if data.get('@type') == 'NewsArticle':
                    article_body = data.get('articleBody',article_body1)
            except json.JSONDecodeError as e:
            # Print the error and problematic JSON data
                print(f'Error decoding JSON: {e}')
                print('Problematic JSON Data:', json_ld_data)
    return article_body

In [ ]:
crime_list=['forge', 'forged', 'forgery', 'drug trafficking', 'drug', 'poaching', 'poacher', 'prostitution', 'trafficking', 'bribe', 'corruption', 'insider trading', 'organ trafficking', 'cyber crime', 'scam', 'chit fund scam', 'it scam', 'nse scam', 'bse scam', 'recruitment scam', 'online scam', 'investment scam', 'smuggling', 'smuggle', 'hair smuggling', 'wildlife smuggling', 'cattle smuggling', 'gold smuggling', 'fraud', 'online fraud', 'cyber fraud', 'bank fraud', 'job fraud', 'gst fraud', 'input tax credit fraud', 'loan fraud', 'housing fraud', 'financial fraud', 'e-fraud', 'crypto fraud', 'cryptocurrency fraud', 'crypto-currency fraud', 'investment fraud', 'extorsion', 'ransom', 'kidnap', 'cheating', 'money-laundering', 'money laundering', 'gst evasion', 'tax evasion', 'terror activity', 'terrorist', 'robbery', 'robber', 'arms racket', 'illegal mining', 'hawala', 'violation', 'murder', 'black marketting', 'riot', 'counterfeit note', 'fake note', 'fake currency', 'counterfeit currency', 'fake notes', 'illegal conversion', 'drug trafficking']

In [ ]:
len(crime_list)
print(len(crime_list))

In [ ]:
lea_regex=re.compile('( \(?DGGI\)? | \(?DRI\)? | \(?IRDA\)? | \(?DVAC\)? | \(?CID\)? | \(?CBI\)? | \(?ED\)? | \(?NIA\)? | \(?CBDT\)? | \(?NCB\)? | \(?SEBI\)? | \(?CBEC\)? | \(?NCB\)? | \(?RBI\)? | \(?MCA\)? | \(?MEA\)? | \(?MHA\)? | \(?DGFT\)? | \(?IB\)? |DIRECTORATE OF ENFORCEMENT|ENFORCEMENT DIRECTORATE|CENTRAL BOARD OF DIRECT TAXES|DIRECTORATE OF REVENUE INTELLIGENCE|DIRECTORATE GENERAL OF GST INTELLIGENCE|CENTRAL BUREAU OF INVESTIGATION|NATIONAL INVESTIGATION AGENCY|NARCOTICS CONTROL BUREAU|SERIOUS FRAUD INVESTIGATION OFFICE|FOREIGNERS DIVISION|CENTRAL ECONOMIC INTELLIGENCE BUREAU|INTELLIGENCE BUREAU|RESEARCH AND ANALYSIS WING|NATIONAL SECURITY COUNCIL SECRETARIAT|RESERVE BANK OF INDIA|SECURITIES AND EXCHANGE BOARD OF INDIA|INSURANCE REGULATORY AND DEVELOPMENT AUTHORITY|BUREAU OF IMMIGRATION|MINISTRY OF EXTERNAL AFFAIRS|MINISTRY OF CORPORATE AFFAIRS|EGMONT|CENTRAL BOARD OF EXCISE AND CUSTOMS|MINISTRY OF HOME AFFAIRS|DIRECTORATE GENERAL OF FOREIGN TRADE|DIRECTORATE OF INCOME TAX|CENTRAL VIGILANCE COMMISSION|DIRECTORATE OF VIGILANCE AND ANTI(-)?CORRUPTION|DIRECTORATE GENERAL OF GST INTELLIGENCE|DIRECTORATE OF REVENUE INTELLIGENCE|INSURANCE REGULATORY AND DEVELOPMENT AUTHORITY)')

rename_lea={'ED': 'ENFORCEMENT DIRECTORATE',
'DIRECTORATE OF ENFORCEMENT': 'ENFORCEMENT DIRECTORATE',
'NIA':'NATIONAL INVESTIGATION AGENCY',
'CBDT':'CENTRAL BOARD OF DIRECT TAXES',
'NCB':'NARCOTICS CONTROL BUREAU' ,
'SEBI': 'SECURITIES AND EXCHANGE BOARD OF INDIA',
'CBEC':'CENTRAL BOARD OF EXCISE AND CUSTOMS' ,
'NCB': 'NARCOTICS CONTROL BUREAU',
'RBI': 'RESERVE BANK OF INDIA',
'MCA': 'MINISTRY OF CORPORATE AFFAIRS',
'MEA':'MINISTRY OF EXTERNAL AFFAIRS',
'MHA': 'MINISTRY OF HOME AFFAIRS',
'DGFT':'DIRECTORATE GENERAL OF FOREIGN TRADE',
'IB': 'INTELLIGENCE BUREAU',
'CBI': 'CENTRAL BUREAU OF INVESTIGATION',
'CID':'CRIMINAL INVESTIGATION DEPARTMENT',
'DVAC': 'DIRECTORATE OF VIGILANCE AND ANTI CORRUPTION',
'DGGI' : 'DIRECTORATE GENERAL OF GST INTELLIGENCE',
'DRI':'DIRECTORATE OF REVENUE INTELLIGENCE',
'IRDA': 'INSURANCE REGULATORY AND DEVELOPMENT AUTHORITY'
           }

In [ ]:
def rename_leas(names):
    #print('*Renaming LEA abbreviations*')
    updated_list=[]
    for i in names:
        if i in rename_lea.keys():
            updated_list.append(rename_lea[i])
        else:
            updated_list.append(i)
    return list(set(updated_list))  

def person_tagger(txt):
    #print('*Tagging Persons*')
    doc=nlp(txt)
    entity=[]
    person_list=[re.sub(r'[^A-Za-z\s]+', '',entity.text.lower().replace('\n','')) for entity in doc.ents if entity.label_=='PERSON']
    person_list=set(person_list)
    return person_list


In [ ]:
def remove_duplicates(df_dups):
    #print('*Removing Duplicates*')
    title_list=list(set(df_dups['title'].to_list()))
    df_dups['cluster_no']=''
    c=1
    processed=[]
    for title in title_list:
        if title not in processed:
            matches=[]
            b=[i for i in title_list if i!=title and i not in processed]
            matches_list=process.extractBests(title, b ,scorer=fuzz.token_sort_ratio,score_cutoff=65)
            if len(matches_list)>=1:
                for i in matches_list:
                    matches.append(i[0])
                matches.append(title)
                #print(c, matches)
                for i in matches:
                    df_dups.loc[df_dups['title']==i,'cluster_no']=c
                c=c+1
                for i in matches:
                    processed.append(i)
            else:
                df_dups.loc[df_dups['title']==title,'cluster_no']=c
                processed.append(title)
                c=c+1
    df_dups['article_length']=df_dups['article'].apply(lambda x: len(nltk.word_tokenize(x)))
    df_dups=df_dups.sort_values(['cluster_no', 'article_length'],ascending=[True,False]).drop_duplicates(['cluster_no'], keep='first')
    df_dups=df_dups[df_dups['article_length']>40].drop(['cluster_no','article_length'],axis=1)
    return df_dups

In [ ]:
regexp=re.compile('^.*//(.*?)/.*$')


In [ ]:
news_providers=['ECONOMICTIMES.INDIATIMES.COM', 'ENGLISH.LOKMAT.COM', 'ANINEWS.IN', 'INDIANEXPRESS.COM', 'TIMESOFINDIA.INDIATIMES.COM', 'HINDUSTANTIMES.COM', 'INDIATODAY.IN', 'NEWINDIANEXPRESS.COM', 'REPUBLICWORLD.COM', 'TIMESNOWNEWS.COM', 'TRIBUNEINDIA.COM', 'THEPRINT.IN', 'BUSINESS-STANDARD.COM', 'BUSINESSTODAY.IN', 'FINANCIALEXPRESS.COM', 'FREEPRESSJOURNAL.IN', 'LIVELAW.IN', 'MONEYCONTROL.COM', 'NEWS18.COM', 'THEHINDUBUSINESSLINE.COM', 'THESTATESMAN.COM', 'UNIINDIA.COM', 'ENGLISH.JAGRAN.COM', 'MUMBAIMIRROR.INDIATIMES.COM', 'NEWS.ABPLIVE.COM', 'SCROLL.IN', 'TEHELKA.COM', 'THEWIRE.IN', 'BUSINESSINSIDER.IN', 'DECCANCHRONICLE.COM', 'DECCANHERALD.COM', 'DNAINDIA.COM', 'INDIATIMES.COM', 'INDIATVNEWS.COM', 'LIVEMINT.COM', 'NDTV.COM', 'NEWS24.COM', 'THEHITAVADA.COM', 'ZEENEWS.INDIA.COM']

In [ ]:
news_providers

In [ ]:
rss_dict=['("Crypto" OR "Cryptocurrency") AND ("scam" OR "fraud" OR "cheating" OR "duped" OR "pyramid")',
'"ransomware" AND ("crore" OR "crores")',
'("China" OR "Chinese") AND "fraud"',
'"Organised crime"',
'"Human trafficking"',
'"Black money"',
'Benami',
'"Sanctions List"',
'"Kashmir" AND "funding"',
'"Separatist" AND "India"',
'"Criminal proceeds"',
'"Denied Persons List"',
'"Visa" AND "Overstaying"',
'"Ponzi" AND "Scheme"',
'"Shell Company" AND "Beneficiary"',
'forge AND (billion OR million OR crores OR crore OR " cr ")',
'trafficking AND (billion OR million OR crores OR crore OR " cr ")',
'bribe AND (billion OR million OR crores OR crore OR " cr ")',
'corruption AND (billion OR million OR crores OR crore OR " cr ")',
'"insider trading" AND (billion OR million OR crores OR crore OR " cr ")',
'smuggle AND (billion OR million OR crores OR crore OR " cr ")',
'ransom AND (billion OR million OR crores OR crore OR " cr ")',
'kidnap AND (billion OR million OR crores OR crore OR " cr ")',
'robbery AND (billion OR million OR crores OR crore OR " cr ")',
'racket AND (billion OR million OR crores OR crore OR " cr ")',
'"illegal mining" AND (billion OR million OR crores OR crore OR " cr ")',
'hawala AND (billion OR million OR crores OR crore OR " cr ")',
'"black marketing" AND (billion OR million OR crores OR crore OR " cr ")',
'"fake currency" AND (billion OR million OR crores OR crore OR " cr ")',
'"counterfeit currency" AND (billion OR million OR crores OR crore OR " cr ")',
'"illegal conversion" AND (billion OR million OR crores OR crore OR " cr ")',
'crime AND (billion OR million OR crores OR crore OR " cr ")',
'scam AND (billion OR million OR crores OR crore OR " cr ")',
'fraud AND (billion OR million OR crores OR crore OR " cr ")',
'evasion AND (billion OR million OR crores OR crore OR " cr ")',
'extortion AND (billion OR million OR crores OR crore OR " cr ")',
'"money laundering" AND (billion OR million OR crores OR crore OR " cr ")',
 #old tags---        
'"Enforcement Directorate" OR "Directorate of Enforcement" OR "ED"',
'"Directorate General of GST Intelligence" OR "DGGI"',
'"Directorate of Revenue Intelligence"',
'"Central Bureau of Investigation" OR "CBI"',
'"National Investigation Agency" OR "NIA"',
'("Income Tax Department" OR "I-T Department" OR "IT Department") AND (Seizure OR seized OR Raid OR fine OR fraud)',
'("Securities and Exchange Board of India" OR "SEBI") AND (Seizure OR seized OR Penalty OR Raid OR fine OR fraud)',
'("Reserve Bank of India" OR "RBI") AND (Penalty OR Raid OR fine OR seizure OR seized OR fraud)',
'("Insurance Regulatory and Development Authority" OR "IRDA") AND (Penalty OR Raid OR fine OR seized OR Seizure OR fraud)',
'"Narcotics Control Bureau" OR "NCB"',
'("GST" OR "Goods and Service Tax") AND (seized OR Seizure OR Customs OR Raid OR fine OR fraud)',
'"Under-invoicing" OR Underinvoicing OR misinvoicing',
'"Over-invoicing" OR Overinvoicing OR misinvoicing',
'"cyber fraud" OR "online fraud" OR "cyber crime" OR cybercrime',
'"cyber police" OR "police special cell"',
'"insurance fraud"',
'"Drug Trafficking"',
'"Illegal mining"',
'"Input Tax Credit Fraud" or "tax credit fraud" or "tax fraud"',
'"Anti Terrorist Squad" OR "Anti-Terrorist Squad" OR "Anti terrorism Squad" OR "Anti-terrorism Squad" OR "anti-terror unit" OR "anti terror unit"',
'"Counterfeit Currency" OR "Fake Indian Currency" OR "Fake currency Racket" OR "Fake currency found"',
'("Wild life" OR Hunting OR Animals OR Wildlife) AND (Rhino OR Smuggl OR syndicate OR racket OR gang OR poaching OR MAFIA)']

In [ ]:
# date_range.to_dict(orient='records')

In [ ]:
# df_news_demo=df_news.head(10)

In [ ]:
# df_final=pd.DataFrame.from_records(fetch_article(df_news_demo))

In [ ]:
# df_final_check=df_final.copy()
# df_news

In [ ]:
for month_row in date_range.to_dict(orient='records'):
    if  month_row['status']!='Done':
        print('Fetching News for Month:',month_row['name'])
        df_news=pd.DataFrame()
        for topic in tqdm.tqdm(rss_dict):
            delay = round(randint(180,360),2)
            print("Sleeping for " + str(round(delay/60,1)) + " minutes")
            time.sleep(round(delay/60,1))
            df_news = df_news._append(pd.DataFrame.from_records(get_titles(topic,month_row['from'],month_row['to'])))
        df_news.drop_duplicates(subset=['title'],inplace=True)
        df_news['selected_news_provider']=df_news['source'].apply(lambda x: str(set([True for i in news_providers if i in x.upper()]))) 
        df_news['selected_news_provider']=df_news['selected_news_provider'].apply(lambda x: 'True' if x=='{True}' else 'False')
        df_news.published = pd.to_datetime(df_news.published)
        df_news['published']=df_news['published'].apply(lambda x: str(x)[0:10])
        df_news=df_news[df_news['selected_news_provider']=='True']
# #         df_news=df_news[['published','title','link','search_criteria','selected_news_provider']]
#         df_final=pd.DataFrame.from_records(fetch_article(df_news))
#         # print(df_final)
#         # df_final['publisher']=df_final['pub'] # extra line added
#         df_final=remove_duplicates(df_final)
#         regexp_quantum=re.compile('((?: Rs|₹|\$)(?:.| )?[0-9]+[0-9.,]*[a-z]?(?: |-)(?:lacs|crores|million|lakh|billion|crore|cr)? ?(?:rupees|rupee|dollars|dollar)?)',re.IGNORECASE)
#         df_final['quantum']=df_final['article'].apply(lambda x: regexp_quantum.findall(str(x),re.IGNORECASE))+df_final['title'].apply(lambda x: regexp_quantum.findall(str(x),re.IGNORECASE))
#         df_final['quantum']=df_final['quantum'].apply(lambda x: list(set([i.replace('₹','Rs ').replace('-','').strip() for i in x ])))
#         df_final['lea']=df_final['article'].apply(lambda x: list(set([i.group().replace('(','').replace(')','').strip() for i in re.finditer(lea_regex, str(x).upper())])))
#         df_final['lea']=df_final['lea'].apply(rename_leas)
#         df_final.dropna(subset=['article'],inplace=True)
#         df_final.dropna(subset=['title'],inplace=True)
#         df_final.drop_duplicates(subset=['title'],inplace=True)
# #         df_final['soucre']=df_news['source'] # extar added line
#         # df_news.drop_duplicates(subset=['source'],inplace=True)
#         df_final=df_final[df_final['title'].apply(lambda x: len(str(x).split()))>4]
#         df_final=df_final[df_final['article'].apply(lambda x: len(str(x).split()))>45]
#         df_final=df_final[~df_final['title'].str.contains('^Nia |(.| )?Nia |^Ed |(.| )?Ed |^ed-|(.| )?ed-')]
#         df_final=df_final[~df_final['article'].str.contains('^Nia |(.| )?Nia |^Ed |(.| )?Ed |^ed-|(.| )?ed-')]
#         df_final['crimes']=df_final['article'].apply(lambda x: [crime for crime in crime_list if crime in x.lower()])+df_final['title'].apply(lambda x: [crime for crime in crime_list if crime in x.lower()])
#         df_final['crimes']=df_final['crimes'].apply(lambda x: list(set(x)))
# #         df_final.to_excel(r'C:\Users\SAL006\FATF\News Files\FATF_News_'+month_row['name']+'.xlsx',index=False)
#         date_range.loc[date_range['from']==month_row['from'],'status']='Done'
# #         date_range.loc[date_range['from']==month_row['from'],'completed_on']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#         df_final.rename(columns={"date":"publish_date"},inplace=True)
############################################################################################################################
#         df_final.to_sql('news', pengine, schema="fatf", if_exists="append", index=False)
#         date_range.to_sql('news_status', pengine, schema="fatf", if_exists="replace", index=False) #arn_no is article number
#         sql="insert into fatf.news23 (arn_no,publish_date, publisher, title, article, persons, keywords, link, search_criteria, quantum, lea, crimes, locations) \
#         SELECT 'ARN'||nextval('fatf.mysequence'),publish_date, publisher, title, article, persons, keywords, link, search_criteria, quantum, lea, crimes, locations \
#         FROM fatf.news where publish_date>='"+date_dict['from'][0]+"'" 
#         conn=pengine.connect()
#         conn.execute(sql)
#         conn.close()

        #date_range.to_csv(r'C:\Users\SAL006\FATF\date_range.csv',index=False)
        #delay = randint(1800,10800)
        ##print("Sleeping for " + str(delay/3600) + "hours")
        ##sleep(delay)
        

In [ ]:
df_final.columns

In [ ]:
df_final.size

# LOCATION EXTRACTION

In [ ]:
def locations(text):
    locations = []
    doc = nlp(text)
    locations.extend(ent.text for ent in doc.ents if ent.label_ in ['LOC'])
    set_location = list((locations))
    return set_location

In [ ]:
df = df_final.copy()

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
df['locations'] = df['article'].progress_apply(locations)

In [ ]:
df.head(1)

In [ ]:
# df.to_excel(f'C:\\Users\\SAL004\\Desktop\\pranav\\weekly adverse media articles\\news_articles_17-04-2023_to_23-04-2023.xlsx', index = False)

In [ ]:
df = df.rename(columns={'pub1':'publisher', 'persons':'personname','searchcriteria':'search_criteria','publish_date':'publishdate'})

In [ ]:
df = df.drop(columns= ['pub1','source'])

In [ ]:
conn = sqlite3.connect(r'C:\Users\SAL004\Desktop\New folder\adverse_media_flask\db2.sqlite')
cursor = conn.cursor()

In [ ]:
cursor.execute("select max(cast(substring(arnno,4,10) as int)) from article2")
max_number = cursor.fetchone()[0]

# Check if max_number is None
if max_number is None:
    max_number = 0  # Assuming you want to start from 1 if the table is empty
else:
    max_number += 1  # Increment the maximum number

# Fetch the rows
cursor.execute("SELECT arnno FROM article2")  # Make sure to execute a new query to fetch rows
rows = cursor.fetchall()

# Display fetched data
# Close the connection
conn.close()

In [ ]:
print(max_number)

In [ ]:
df.insert(0, 'arnno', [max_number + i for i in range(len(df))])

In [ ]:
df['arnno'] = df['arnno'].astype(str)

In [ ]:
df['arnno'] = df['arnno'].apply(lambda x: 'ARN' + x)

In [ ]:
df.to_excel(f'C:\\Users\\SAL004\\Desktop\\pranav\\weekly adverse media articles\\news_articles_{date_before}_to_{today}.xlsx', index = False)

In [ ]:
df['title'] = df['title'].astype(str)
df['publisher'] = df['publisher'].astype(str)
df['article'] = df['article'].astype(str)
df['personname'] = df['personname'].astype(str)
df['keywords'] = df['keywords'].astype(str) 
df['link'] = df['link'].astype(str)
df['search_criteria'] = df['search_criteria'].astype(str)
df['quantum'] = df['quantum'].astype(str)
df['lea'] = df['lea'].astype(str)
df['crimes'] = df['crimes'].astype(str)
df['locations'] = df['locations'].astype(str)

In [ ]:
# df = df.drop(columns=['publish_date'])

In [ ]:
df.head(1)

In [ ]:
conn = sqlite3.connect(r'C:\\Users\\SAL004\\Desktop\\New folder\\adverse_media_flask\\db2.sqlite')
df.to_sql('article2', conn, if_exists='append', index=False)

### LAST STEP, UPDATE DATE ABOVE, ABOVE IS THE LAST COLUMN

# Month End and Next month beginning duplicates removal

In [ ]:
# date_range=pd.read_csv(r'C:\Users\SAL006\FATF\date_range_15days_window.csv')

In [ ]:
# conn=pengine.connect()
# for row in date_range.to_dict('rows'):
#     sql="SELECT * from fatf.news where publish_date between '"+ row['from'] +"' and '"+ row['to'] +"'"
#     data_table=pd.read_sql_query(sql,conn)
#     dedup_data=remove_duplicates(data_table)
#     dedup_data.to_sql('news_final', pengine, schema="fatf", if_exists="append", index=False)
# conn.close()

In [8]:
df_news

NameError: name 'df_news' is not defined

In [13]:
df_news=pd.read_csv(r'C:\Users\LTIM_10700357\Downloads\test_news.csv')
df_news_demo = df_news.head(10)

In [14]:
# df_final=pd.DataFrame.from_records(fetch_article(df_news_demo))

In [15]:
df_final = pd.DataFrame()

In [19]:
import json
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

article_body1 = 'No Records Extracted'
url='https://news.google.com/rss/articles/CBMitAFBVV95cUxQSm5KTktQVGs5eTBTYkdmR2FwNHQ4eEVwbGpiclRxWno4cS1FNDlqWXJTQlVqUXB4RktPTG9IZl94aTRaNlZ3QzlCSWFwVW9jeUdTRTlIcGdjQlptSDVPWkJsQV9iTERRaWw5TGhaSXRmOU41eXF3LTZRZ2NSZmE0MEl2Mk16RDFHenk4LUI4Z3d4bTMySzBrMEdxUGhWaHNlaWJRazBJRllwa1lTRmFJRjhGZF_SAbwBQVVfeXFMTXdhbHdSSXU4eVhCakZvMW5sUWRjdTd1OHBrS2FWandsb2tFN1hCVV94Nk5yZDBQelM1Wjk4clJiNUQ0eEdSRmtYaGQzcnFsTXh2d2Q4VndPLUtVWXhnWjFvckZtb0NNY2g5dnpxbnh0U2Z2cGhobnVzZVNBdkVYck9KaEctRTVCamw0ZEpkWkFoZEo0V1A4WVI5bWQ1Z3NBMnhneVk3SDZsOUhLUlhmUUhHdV9FaTJPTWh3TTg?oc=5'
webdriver_path = r'C:\Users\LTIM_10700357\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe'  # Adjust this path
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
# Initialize WebDriver
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    driver.get(url)
#        sleep(randint(10, 100))  # Random delay to mimic human behavior
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    scripts = soup.find_all('script', type='application/ld+json')
    
    for script in scripts:
        if script:
            json_ld_data = script.string.strip().replace('\n', '').replace('  ', '')
            try:
                data = json.loads(json_ld_data)
                if data.get('@type') == 'NewsArticle':
                    article_body = data.get('articleBody', article_body1)
                    return article_body  # Return immediately if article body is found
            except json.JSONDecodeError as e:
                print(f'Error decoding JSON: {e}')
                print('Problematic JSON Data:', json_ld_data)
except Exception as e:
    print(f"An error occurred: {e}")

finally:
        driver.quit()  # Ensure the driver quits regardless of success or failure


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 125
Current browser version is 129.0.6668.90 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF671A322C2+60002]
	(No symbol) [0x00007FF6719ACA59]
	(No symbol) [0x00007FF671867EDA]
	(No symbol) [0x00007FF6718A6242]
	(No symbol) [0x00007FF6718A5418]
	(No symbol) [0x00007FF67189F007]
	(No symbol) [0x00007FF67189AFA4]
	(No symbol) [0x00007FF6718E79A4]
	(No symbol) [0x00007FF6718E7000]
	(No symbol) [0x00007FF6718DC033]
	(No symbol) [0x00007FF6718A9657]
	(No symbol) [0x00007FF6718AA251]
	GetHandleVerifier [0x00007FF671D43E2D+3278285]
	GetHandleVerifier [0x00007FF671D90190+3590448]
	GetHandleVerifier [0x00007FF671D861D0+3549552]
	GetHandleVerifier [0x00007FF671AE1DE6+779654]
	(No symbol) [0x00007FF6719B7ACF]
	(No symbol) [0x00007FF6719B2EE4]
	(No symbol) [0x00007FF6719B3072]
	(No symbol) [0x00007FF6719A2C4F]
	BaseThre

In [17]:
for k in df_news_demo['link']:
    print(k)

https://news.google.com/rss/articles/CBMitAFBVV95cUxQSm5KTktQVGs5eTBTYkdmR2FwNHQ4eEVwbGpiclRxWno4cS1FNDlqWXJTQlVqUXB4RktPTG9IZl94aTRaNlZ3QzlCSWFwVW9jeUdTRTlIcGdjQlptSDVPWkJsQV9iTERRaWw5TGhaSXRmOU41eXF3LTZRZ2NSZmE0MEl2Mk16RDFHenk4LUI4Z3d4bTMySzBrMEdxUGhWaHNlaWJRazBJRllwa1lTRmFJRjhGZF_SAbwBQVVfeXFMTXdhbHdSSXU4eVhCakZvMW5sUWRjdTd1OHBrS2FWandsb2tFN1hCVV94Nk5yZDBQelM1Wjk4clJiNUQ0eEdSRmtYaGQzcnFsTXh2d2Q4VndPLUtVWXhnWjFvckZtb0NNY2g5dnpxbnh0U2Z2cGhobnVzZVNBdkVYck9KaEctRTVCamw0ZEpkWkFoZEo0V1A4WVI5bWQ1Z3NBMnhneVk3SDZsOUhLUlhmUUhHdV9FaTJPTWh3TTg?oc=5
https://news.google.com/rss/articles/CBMi8AFBVV95cUxNZGZhNkN0dHhwMzgzcWd6Y2dZZFBKOWliTUxzS2tPaUtFYmFzWWlwckZZS2lRa29qUldpTzRTYkpOWTctNnFRaV9PazlZcnJoOW5oTlBXbHE2UzhGb3RmVFprdG5zSFNBY0N2eDQtTjRjM28wNEkzZjhsZXQwQUZfalpCZlJWRmd6MXZMZ01vZjZBSmFsNTAzek1JQ3ZMMGRHMG5pYUtrcE1fM1RJRi1QOHF1RnhBTDRYTjI5dmVTTk9wVXVOdUZWaERNbVVjTU5Kek42UEZMXzBTZG1zbzIzZ0lIUnQ4Mk9kUjNyZTNFemzSAfYBQVVfeXFMTXBWWk9aR25vWHZmYjdSMEdiMjBXWmFmNUk5SjBwWHdITGZjS0xLLTM5cVh2RkQ3d0V3ODd4QlNmd